# 第6章 Function Calling与Agent 智能体

# 学习目标

1.理解Function Calling的概念  
2.理解Function Calling的工作原理  
3. 实战使用OpenAI提供的Function Calling接口（基础请求及优化）  
4.探讨自定义Function的提供的可能性  
5. 探讨Function Calling在大模型应用场景中带来的“质变”  
6. 智能体/LLM应用的定义与作用

- 什么是智能体？智能体与大模型的关系是什么样的  
- 智能体概念的演进过程, 基本架构与功能

7. 智能体开发框架smolagents  
8. Agentic RAG 实战  
9. 智能体的应用场景

- 探讨智能体在行业场景中的落地情况  
- 探讨智能体系统/LLM应用的常见分类

# 1. Function Calling的概念

Function Calling (函数调用), 顾名思义, 为模型提供了一种调用函数的方法/能力。

- Function Calling成立的模型能力基础：

问题理解和行动规划  
结构化数据输出  
- 上下文学习 In-Context Learning

Function Calling让模型输出不再局限于自身推理输出，而是可以与外部系统交互，完成更复杂的任务

- 常见Function Calling应用场景包括：

- 查询检索，补充额外信息（如RAG、搜索）  
- 理解用户输入，向外部系统写入信息（如表单填写）  
- 调用外部系统能力，完成实际行动为动作（如下订单）

# 2. Function Calling的工作原理

# 2.1 OpenAI官方定义

OpenAI官方说明文档：https://platform.openai.com/docs/guides/function-calling

# 工具调用流程

工具调用是通过 OpenAI API 在应用程序和模型之间进行的多步骤对话。工具调用流程包含五个高级步骤：

1. 使用模型可以调用的工具向模型发出请求  
2. 接收来自模型的工具调用  
3. 使用工具调用的输入在应用程序端执行代码  
4. 使用工具输出向模型发出第二个请求  
5. 接收来自模型的最终响应（或更多工具调用）


![](./img/1.png)

![](./img/2.png)

# 2.3 Calling是结果，理解和选择才是第一步

- 除了代表用户诉求的Prompt之外，Function Calling还需要将可用的工具信息（Function Definitions）也提供给模型  
在第一次请求时，模型的核心工作如下：

1. 理解Prompt所代表的“诉求”和Definitions所代表的“行动可能性”  
2. “选择”完成“诉求”所需要进行的“行动”（从“行动可能性”中获得）  
3. 根据所选择的“行动”，给出执行“行动”所需的“行动参数”（Parameters）

- 那么想一想：

1. 什么影响“选择”的效果？  
2. 什么影响“行动”的可执行性和效果？


# 2.4 作为可选项的结果回调和最终回复输出

- 在对话流中, 将Function Calling的结果 (Function Result) 与初始的Prompt诉求再次组合, 提供给模型以获得最终的回复输出, 是常见的流程 (RAG就是一个典型的例子)  
- 但如果我们将Function Calling用于非对话流场景，最终回复输出就不一定是必选项了，例如：

1.【只需要完成Calling动作】我们只是希望通过Function Calling完成行动选择和发起，接下来就进入业务处理流程，例如：理解用户表达并代替用户下单  
2.【只需要完成行动参数Parameters生成】我们只是希望将Function Calling做好工具使用决策，并完成部分请求参数的生成，接下来需要走业务流程补全其他参数（比如鉴权信息），例如：敏感数据查询

在实际生产中，不给出最终回复输出，而只是使用Function Calling返回的调用方法数据，是很常见的用法。

# 3 实际调用Function Calling

## 从官方案例开始

## 第一步 工具决策与调用信息生成


In [24]:
from openai import OpenAI

client = OpenAI(
    api_key="sk-66bc27a6330f434f8751f8172a73064f",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# 工具定义
tools = [
    {
        #类型(固定格式)
        "type":"function",
        # 函数定义
        "function":{
            # 函数名称(帮助我们去查找本地的函数在哪里，函数映射的ID)
            "name":"get_weather",
            #函数描述(帮助模型理解函数的作用，适用场景，可以理解为Prompt的一部分)
            "description":"获取提供坐标的当前温度(摄氏度)",
            # 函数依赖的参数的定义(帮助模型理解如果要做参数生成，应该怎么生成)
            "parameters":{
                #参数形式
                "type":"object",# 对应输出JSON string
                # 参数结构
                "properties":{
                    #参数名称
                    "latitude":{"type":"number"},
                    "longlitude":{"type":"number"}
                },
                      #必须保证生成的参数列表
            "required":["latitude","longlitude"],
            "additionalProperties":False
            },
            # 格式是否严格
            "strict":True
        }
    }
]

messages = [{"role":"user","content":"今天上海的天气怎么样？"}]
#messages = [{"role":"user","content":"你今天怎么样？"}]
#messages = [{"role":"user","content":"请告诉我北京的经纬度"}]
# messages = [{"role":"user","content":"今天天气怎么样？"}]

completion = client.chat.completions.create(
    model="qwen3-coder-plus",
    messages=messages,
    tools=tools
)

if completion.choices[0].message.tool_calls:
    print(completion.choices[0].message.tool_calls[0].function)
else:
    print("没有函数调用")



Function(arguments='{"latitude": 31.2304, "longlitude": 121.4737}', name='get_weather')


## 第二步：时间调用工具




In [25]:

function_calling_message = completion.choices[0].message
function_calling = completion.choices[0].message.tool_calls[0]
print("Call Function Name :",function_calling.function.name)
print("Call Function Arguments :",function_calling.function.arguments)

Call Function Name : get_weather
Call Function Arguments : {"latitude": 31.2304, "longlitude": 121.4737}


In [26]:
import json

def get_weather(*,latitude:float,longlitude:float):
    return {
        "temperature":23,
        "weather":"晴天",
        "wind_direction":"东南风",
        "windy":2
    }
functions = {
    "get_weather":get_weather
}    

function_res = functions[function_calling.function.name](**json.loads(function_calling.function.arguments))
print(function_res)


{'temperature': 23, 'weather': '晴天', 'wind_direction': '东南风', 'windy': 2}


In [29]:
# 必须：让模型知道自己之前给了一个什么指令（包含 tool_call_id）

messages.append(function_calling_message)
# 包含了tool_call_id 的结果加入消息队列
messages.append(
   {
        "role":"tool",
        "tool_call_id":function_calling.id,
        "content":str(function_res)
   }
)
# print(messages)


final_res = client.chat.completions.create(
    model="qwen3-coder-plus",
    messages=messages,
    tools=tools
)
# print(fiallnal_res)

print(final_res.choices[0].message.content)



今天上海的天气是晴天，温度为23摄氏度，吹的是东南风，风力为2级。非常适合外出活动！


# 调用tools 失败情况下

In [ ]:
import json
from openai import OpenAI

client = OpenAI(
    api_key="sk-66bc27a6330f434f8751f8172a73064f",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# 工具定义
tools = [
    {
        #类型(固定格式)
        "type":"function",
        # 函数定义
        "function":{
            # 函数名称(帮助我们去查找本地的函数在哪里，函数映射的ID)
            "name":"get_weather",
            #函数描述(帮助模型理解函数的作用，适用场景，可以理解为Prompt的一部分)
            "description":"获取提供坐标的当前温度(摄氏度)",
            # 函数依赖的参数的定义(帮助模型理解如果要做参数生成，应该怎么生成)
            "parameters":{
                #参数形式
                "type":"object",# 对应输出JSON string
                # 参数结构
                "properties":{
                    #参数名称
                    "latitude":{"type":"number"},
                    "longlitude":{"type":"number"}
                },
                      #必须保证生成的参数列表
            "required":["latitude","longlitude"],
            "additionalProperties":False
            },
            # 格式是否严格
            "strict":True
        }
    }
]

messages = [{"role":"user","content":"今天上海的天气怎么样？"}]
#messages = [{"role":"user","content":"你今天怎么样？"}]
#messages = [{"role":"user","content":"请告诉我北京的经纬度"}]
# messages = [{"role":"user","content":"今天天气怎么样？"}]

completion = client.chat.completions.create(
    model="qwen3-coder-plus",
    messages=messages,
    tools=tools
)

if completion.choices[0].message.tool_calls:
    print(completion.choices[0].message.tool_calls[0].function)
else:
    print("没有函数调用")

function_calling_message = completion.choices[0].message
function_calling = completion.choices[0].message.tool_calls[0]
# print("Call Function Name :",function_calling.function.name)
# print("Call Function Arguments :",function_calling.function.arguments)

def get_weather(*,latitude:float,longlitude:float):
    return {
        "temperature":23,
        "weather":"晴天",
        "wind_direction":"东南风",
        "windy":2
    }
functions = {
    "get_weather":get_weather
}

# function_res = functions[function_calling.function.name](**json.loads(function_calling.function.arguments))
# 返回错误信息
function_res = {"c":"键 'latitude' 已不再支持，请改用 'lat'。"}
# print(function_res)

# 必须：让模型知道自己之前给了一个什么指令（包含 tool_call_id）
error_messages = messages[:1]
error_messages.append(function_calling_message)
# 包含了tool_call_id 的结果加入消息队列
error_messages.append(
   {
        "role":"tool",
        "tool_call_id":function_calling.id,
        "content":str(TypeError(function_res))
   }
)
# print(messages)

final_res = client.chat.completions.create(
    model="qwen3-coder-plus",
    messages=error_messages,
    tools=tools
)
# print(final_res)

print(final_res.choices[0].message.content)


# 4 在实际应用场景中一些案例
## 4.1 封装基本方法

In [58]:
import requests
import json
from typing import  TypedDict
from openai import OpenAI
from openai.types.chat import ChatCompletionMessageToolCall

class FunctionCallingResult(TypedDict):
    name: str
    arguments: str

class ModelRequestWithFunctionCalling:
    def __init__(self):
        self._client = OpenAI(
            api_key="sk-66bc27a6330f434f8751f8172a73064f",
            base_url="https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
        self._function_infos = {}
        self._function_mappings = {}
        self._messages = []
    def register_function(self,*,name,description,parameters,function,**kwargs):
        self._function_infos.update(
            {
                name:{
                    "type":"function",
                    "function":{
                        "name":name,
                        "description":description,
                        "parameters":parameters,
                        **kwargs,
                    },
                }
            }
        )
        self._function_mappings.update({name:function})
        return self

    def reset_messages(self):
        self._messages = []
        return self

    def append_message(self,role,content,tool_calls=None):
        msg = {
            "role": role,
            "content": content or "",  # 确保content不为None
        }
        # 仅当tool_calls存在且非空时添加，且转为字典格式
        if tool_calls:
            msg["tool_calls"] = [self._tool_call_to_dict(tc) for tc in tool_calls]
        self._messages.append(msg)
        print("[Processing Messages]:", self._messages)
        return self
    # 辅助方法：将ToolCall对象转为可序列化的字典
    def _tool_call_to_dict(self, tool_call: ChatCompletionMessageToolCall) -> dict:
        return {
            "id": tool_call.id,
            "type": tool_call.type,
            "function": {
                "name": tool_call.function.name,
                "arguments": tool_call.function.arguments,
            }
        }
    def _call(self,function_call_result: FunctionCallingResult):
        function = self._function_mappings[function_call_result.name]
        arguments = json.loads(function_call_result.arguments)
        return function(**arguments)
    def request(self,*,role="user",content=None):
        if role and content:
            self._messages.append({
                "role":role,
                "content":content,
            })
        result = self._client.chat.completions.create(
            model="qwen3-coder-plus",
            messages=self._messages,
            tools=self._function_infos.values()
        )
        self.append_message(**dict(result.choices[0].message))
        if result.choices[0].message.tool_calls:
            for tool_call in result.choices[0].message.tool_calls:
                call_result = self._call(tool_call.function)
                self.append_message("tool",str(call_result),tool_call_id=tool_call.id)
                return self.request()
        else:
            self.append_message("assistant",result.choices[0].message.content)
            return result.choices[0].message.content



## 4.2 联网检索显示场景

In [59]:
import requests
import json
import os
from typing import  TypedDict
from openai import OpenAI
from openai.types.chat import ChatCompletionMessageToolCall

amap_key = os.getenv("Gaode_KEY")
amap_url="https://restapi.amap.com/v5"

def get_loaction_coordinate(location,city):
    url = f"{amap_url}/place/text?key={amap_key}&keywords={location}&region={city}"
    r = requests.get(url)
    result = r.json()
    if "pois" in result and result["pois"]:
        return result["pois"][0]
    return None

def search_nearby_pois(longitude,latitude,keyword):
    url = f"{amap_url}/place/around?key={amap_key}&keywords={keyword}&location={longitude},{latitude}"
    r = requests.get(url)
    result = r.json()
    ans = ""
    if "pois" in result["pois"] and result["pois"]:
        for i in range(min(3,len(result["pois"]))):
            name = result["pois"][i]["name"]
            address = result["pois"][i]["address"]
            distance = result["pois"][i]["distance"]
            ans += f"{name}\n {address}\n 距离 :{distance}米 \n\n"

    return ans

function_calling_request = ModelRequestWithFunctionCalling()

function_calling_request.register_function(
        name="get_loaction_coordinate",
        description="根据 POI 名称,获得 POI的经纬度坐标",
        parameters={
            "type":"object",
            "properties":{
                "location":{
                    "type":"string",
                    "description":"POI名称，必须是中文"
                },
                "city":{
                    "type":"string",
                    "description":"POI所在的城市，必须是中文"
                }
            },
            "required":["location","city"]
        },
        function=get_loaction_coordinate,
    ).register_function(
        name="search_nearby_pois",
        description="搜索给定坐标附近的 POI",
        parameters={
            "type": "object",
            "properties": {
                "longitude": {
                    "type": "string",
                    "description": "中心点的经度"
                },
                "latitude": {
                    "type": "string",
                    "description": "中心点的纬度"
                },
                "keyword": {
                    "type": "string",
                    "description": "目标 POI的关键字"
                }
            },
            "required": ["longitude", "latitude","keyword"]
        },
        function=search_nearby_pois,
)
result = function_calling_request.request(content="五道口附近的咖啡馆")
print("-------------\n\n",result)


TypeError: ModelRequestWithFunctionCalling.append_message() got an unexpected keyword argument 'refusal'

# 4.3 利用文心4.0以上模型作为搜索工具

In [56]:
from openai import OpenAI

client = OpenAI(
    api_key="bce-v3/ALTAK-H6EzzqIaVN5d3ZgeCvf3U/6de527fe81d2b6d37692f296198565368d676db4",  # 千帆bearer token
    base_url="https://qianfan.gz.baidubce.com/v2",  # 千帆域名
)

completion = client.chat.completions.create(
    model="ernie-4.5-turbo-32k", # 预置服务请查看模型列表，定制服务请填入API地址
    messages=[{'role': 'user', 'content': '今天头条热搜榜有哪些新闻？'}],
    stream=True,
    extra_body={
         "web_search":{
            "penalty":1,
            "enable":True
    }
    },
    max_completion_tokens=12288,
    temperature=0.95,top_p=0.7
)

#print(completion.choices[0].message.content)

for chunk in completion:
    if hasattr(chunk.choices[0].delta,"reasoning_content") and  chunk.choices[0].delta.reasoning_content:
        print(chunk.choices[0].delta.reasoning_content,end="",flush=True)
    else:
        print(chunk.choices[0].delta.content,end="",flush=True)
    

根据最新信息，2025年12月13日今日头条热搜榜上的新闻涵盖了历史纪念、社会民生、国际冲突、娱乐动态等多个领域，以下是部分热搜新闻的归纳：

### 一、历史纪念与国家行动

* **南京大屠杀死难者国家公祭日**：2025年12月13日是南京大屠杀死难者国家公祭日，上午10时在侵华日军南京大屠杀遇难同胞纪念馆举行国家公祭仪式，南京全城鸣响防空警报，提醒人们铭记历史、珍爱和平。

### 二、社会民生与政策动态

* **全国粮食总产量增长**：2025年全国粮食总产量达14298亿斤，比去年增长1.2%，彰显农业稳产保供能力。
* **市场监管新规**：国家市场监督管理总局研究起草《汽车行业价格行为合规指南（征求意见稿）》，聚焦新车销售环节，规范明码标价、虚假促销等行为；同时起草《广告引证内容执法指南（征求意见稿）》，回应广告中“销量第一”等违法使用绝对化用语问题。
* **北京地铁新线通车**：北京地铁18号线本月底通车，串联天通苑、回龙观与上地软件园，全程仅需约半小时，将有效缓解区域交通压力。
* **北京整治网络名人乱象**：北京市网信办启动专项行动，整治网络名人账号煽动对立、宣扬不良价值观等问题，关闭劣迹网红“东北雨姐”“猫一杯”等转世账号。

### 三、国际冲突与外交动态

* **泰柬边境冲突**：泰国与柬埔寨边境地区爆发冲突，致大量民众撤离，泰国开放临时避难点安置超17万人，国际社会呼吁双方对话缓和局势。
* **美日关系动态**：日本被指在台湾问题上配合美国战略，却遭美国加征关税、限制光刻胶出口等反制措施，暴露盟友间利益分歧。

### 四、娱乐与文化动态

* **短剧《家里家外2》爆红**：该剧上线3天播放量破10亿，连续6天热度值过亿，成为现象级作品，其温情叙事引发全民共情。
* **其他娱乐话题**：包括明星动态、影视音乐作品讨论等，如周深演唱的短剧主题曲《时间啊》登上QQ音乐流行指数榜。

# 5. Function Calling在大模型应用场景中带来的“质变”

知识层面：从模型自身知识（来源于训练语料）扩展到真实世界知识  
- 行为层面：从“思考模拟器”、“问题应答”扩展到“理解问题-选择行动-发起请求-理解结果-给出回应”  
- 架构层面：让模型不再是一个孤立模块，而是可以融入现有信息系统之中

给软件开发思想带来的冲击：

- 不是基于“规则”而是基于“世界理解”的调用  
- 接纳没有明确的处理过程带来的输出不确定性 (如数据查询)  
- 不走极端：“全盘拒绝”和“全盘接受”都不可取


# 6. 智能体/LLM应用的定义

- Agent智能体的概念存在“过度炒作”的现象，部分媒体使用“智能体”这个词指代任何基于LLM能力构建的应用  
- 不给出清晰定义的概念讨论甚至衍生讨论都是耍流氓

# 6.1 机器学习概念中的Agent

在机器学习领域，智能体（Agent）通常指能够感知环境、做出决策并采取行动以实现特定目标的实体。这些智能体具备自主性，能够通过传感器获取环境信息，经过内部处理后，通过执行器对环境施加影响。这种架构使智能体能够在复杂、多变的环境中自主运作。

例如：

- Alpha Go  
- 星际争霸/Dota 2对战AI  
学踢足球的AI点击观看

# 6.2 由各类开源项目实践并由Lilian Weng总结的LLM-Powered Autonomous Agents

- 原文地址：https://lilianweng.github.io/posts/2023-06-23-agent/  
- 重要意义：给基于LLM驱动的智能体讨论提供了共识性的基础定义  
- 架构图：


![](./img/3.png)


# - 核心概念：

- 核心驱动：LLM（提供基础智力、通识、逻辑、上下文内学习等基础能力）  
- 关键组件：

- 规划（Planning）：将复杂任务分解为可管理的子目标（Task Decomposition），并通过自我反思（Self-Reflection）来提高结果质量  
。记忆（Memory）：包括短期记忆（对话记录）和长期记忆（通过外部向量存储和快速检索来保留和回忆信息）（这部分突破项目不多，去年有一个叫Mem0的项目刷过一次屏)  
。工具使用（Tool Use）：学习调用外部工具，补充额外信息或完成环境交互

- Inspiration:

- [ReAct论文] (https://arxiv.org/abs/2210.03629)  
AutoGPT  
GPT-Engineer  
BabyAGI


# 6.3 多智能体协同

- 智能体概念的提出让一批使用类似上述结构（通常是简化的结构，比如只使用Role设定，或是ReAct Prompt）尝试进行多次模型请求协同的项目被关注，核心思想是通过不同的智能体分工协作，组成更大的协作网络  
- 代表项目：

- Camel.ai  
- MetaGPT  
- Microsoft AutoGen  
- OpenAI Swarm(现在的Agent SDK)
- 秒哒（国内）

# 7. 智能体开发框架smolagents

# 7.1 smolagents 介绍

GibHub: https://github.com/huggingface/smolagents

官方文档：https://hugging.acce.co/docs/smolagents/index

smolagents是HuggingFace官方推出的Agent开发库，构建强大 agent 的最简单框架！。

首先来介绍一下smolagents吧，smol是small的俏皮用法，故smolagents的含义是“轻量的agent工具”。smolagents库提供：

- 简洁性：Agent逻辑仅需约千行代码。我们将抽象保持在原始代码之上的最小形态！

- 支持任何LLM：支持通过Hub托管的模型，使用其transformers版本或通过我们的推理API加载，也支持OpenAI、Anthropic等模型。使用任何LLM为agent提供动力都非常容易。

- 一流的代码 agent 支持，即编写代码作为其操作的 agent（与“用于编写代码的 agent”相对），在此了解更多。

- Hub 集成：您可以在 Hub 上共享和加载工具，更多功能即将推出！


# 8 实战：Agentic RAG

# 8.1 传统RAG的局限性

尽管传统 RAG 方法有诸多优势,但它也面临一些挑战：

1. 单次检索步骤：如果初始检索结果较差,则最终生成的结果将受到影响  
2. 查询文档不匹配：用户查询（通常是问题）可能与包含答案（通常是陈述）的文档不太匹配  
3. 推理能力有限：简单的 RAG 流程无法进行多步推理或查询细化  
4. 上下文窗口约束：检索到的文档必须适合模型的上下文窗口

![](./img/4.png)

![](./img/5.png)

![](./img/6.png)

你需要一个有效的 token 作为环境变量 HF_TOKEN 来调用 Inference Providers。

Hugging Face注册-登录-创建Access Tokens-系统环境变量配置HF_TOKEN

我们首先加载一个知识库以在其上执行 RAG：此数据集是许多 Hugging Face 库的文档页面的汇编，存储为 markdown 格式。我们将仅保留 transformers 库的文档。然后通过处理数据集并将其存储到向量数据库中，为检索器准备知识库。我们将使用 LangChain 来利用其出色的向量数据库工具。



In [60]:
import os
from dotenv import load_dotenv
load_dotenv()

import datasets
from langchain_core.documents import Document
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_classic.retrievers import BM25Retriever

knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]}) for doc in knowledge_base
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)
print(docs_processed)


/opt/anaconda3/envs/6_functionCalling_Agent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



现在文档已准备好。我们来一起构建我们的 agent RAG 系统！我们只需要一个 RetrieverTool，我们的 agent 可以利用它从知识库中检索信息。

由于我们需要将 vectordb 添加为工具的属性，我们不能简单地使用带有 @tool 装饰器的简单工具构造函数：因此我们将遵循 tools 教程中突出显示的高级设置。

In [61]:

from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses lexical search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be lexically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(docs, k=10)

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved documents:\n" + "".join(
            [f"\n\n===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
        )


BM25 检索方法是一个经典的检索方法，因为它的设置速度非常快。为了提高检索准确性，你可以使用语义搜索，使用文档的向量表
示替换 BM25:因此你可以前往 MTEB Leaderboard 选择一个好的嵌入模型。

现在我们已经创建了一个可以从知识库中检索信息的工具，现在我们可以很容易地创建一个利用这个 retriever_tool 的 agent！因此 agent 将使用如下参数初始化：

- tools：代理将能够调用的工具列表。  
- model：为代理提供动力的LLM。

我们的 model 必须是一个可调用对象，它接受一个消息的 list 作为输入，并返回文本。它还需要接受一个 stop_sequences 参数数，指示何时停止生成。为了方便起见，我们直接使用包中提供的 HfEngine 类来获取调用 Hugging Face 的 Inference API 的 ALLM 引擎。

接着，我们将使用 meta-llama/Llama-3.3-70B-Instruct 作为 llm 引擎，因为：

- 它有一个长  $128 \mathrm{k}$  上下文，这对处理长源文档很有用。  
- 它在 HF 的 Inference API 上始终免费提供！





In [62]:

from smolagents import CodeAgent, InferenceClientModel

retriever_tool = RetrieverTool(docs_processed)
agent = CodeAgent(
    tools=[retriever_tool],
    model=InferenceClientModel(model_id="Qwen/Qwen3-Next-80B-A3B-Thinking",api_key=os.getenv("HuggingFace_KEY")),
    max_steps=4,
    verbosity_level=2,
    stream_outputs=True,
)


当我们初始化 CodeAgent 时，它已经自动获得了一个默认的系统提示，告诉 LLM 引擎按步骤处理并生成工具调用作为代码片段但你可以根据需要替换此提示模板。接着，当其.run()方法被调用时，代理将负责调用 LLM 引擎，并在循环中执行工具调用直到工具 final answer 被调用，而其参数为最终答案。

In [63]:

agent_output = agent.run("For a transformers model training, which is slower, the forward or the backward pass?")

print("Final output:")
print(agent_output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ For a transformers model training, which is slower, the forward or the backward pass?                           │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Next-80B-A3B-Thinking ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/opt/anaconda3/envs/6_functionCalling_Agent/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  query = "transformers model training forward backward pass speed comparison"                                     
  result = retriever(query=query)                                                                                  
  print(result)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:

Retrieved documents:


===== Document 0 =====
Saving all activations from the forward pass in order to compute the gradients during the backward pass can result 
in 
significant memory overhead. The alternative approach of discarding the activations and recalculating them when 
needed 
during the backward pass, would introduce a considerable computational overhead and slow down the training process.

===== Document 1 =====
- A train step function which combines the loss function and optimizer update, does the forward and backward pass 
and returns the updated parameters.

===== Document 2 =====
overhead. This is super helpful when you have activation checkpointing enabled, where we do a forward recompute and
backward passes a single layer granularity and want to keep the parameter in the forward recompute till the 
backward

===== Document 3 =====
For convolutions and linear layers there are 2x flops in the backward compared to the forward, which generally 
translates 
into ~2x slower (sometimes more, because sizes in the backward tend to be more awkward). Activations are usually 
bandwidth-limited, and it’s typical for an activation to have to read more data in the backward than in the forward
(e.g. activation forward reads once, writes once, activation backward reads twice, gradOutput and output of the 
forward,

===== Document 4 =====
becomes possible to increase the **effective batch size** beyond the limitations imposed by the GPU's memory 
capacity. 
However, it is important to note that the additional forward and backward passes introduced by gradient 
accumulation can 
slow down the training process.

===== Document 5 =====
## How to benchmark 🤗 Transformers models

The classes [`PyTorchBenchmark`] and [`TensorFlowBenchmark`] allow to flexibly benchmark 🤗 Transformers models. 
The benchmark classes allow us to measure the _peak memory usage_ and _required time_ for both _inference_ and 
_training_.

<Tip>

Hereby, _inference_ is defined by a single forward pass, and _training_ is defined by a single forward pass and
backward pass.

</Tip>

===== Document 6 =====
than 10 seconds. In case only very large checkpoints are available,
    it might make more sense to create a dummy model in the new
    environment with randomly initialized weights and save those weights
    for comparison with the 🤗 Transformers version of your model
-   Make sure you are using the easiest way of calling a forward pass in
    the original repository. Ideally, you want to find the function in
    the original repository that **only** calls a single forward pass,

===== Document 7 =====
than 10 seconds. In case only very large checkpoints are available,
    it might make more sense to create a dummy model in the new
    environment with randomly initialized weights and save those weights
    for comparison with the 🤗 Transformers version of your model
-   Make sure you are using the easiest way of calling a forward pass in
    the original repository. Ideally, you want to find the function in
    the original repository that **only** calls a single forward pass,

===== Document 8 =====
In case only very large checkpoints are available, it might make more sense to create a dummy model in the new
  environment with randomly initialized weights and save those weights for comparison with the 🤗 Transformers 
version
  of your model
- Make sure you are using the easiest way of calling a forward pass in the original repository. Ideally, you want 
to
  find the function in the original repository that **only** calls a single forward pass, *i.e.* that is often 
called

===== Document 9 =====
```python
model = BarkModel.from_pretrained("suno/bark-small", torch_dtype=torch.float16, 
attn_implementation="flash_attention_2").to(device)
```

##### Performance comparison

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("backward")                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: backward

[Step 2: Duration 6.05 seconds| Input tokens: 5,149 | Output tokens: 1,290]

Final output:
backward


# 9. 智能体系统/LLM应用的应用场景

# 项目举例：

- DeepResearch

# 应用场景举例：

- 文案生成  
- AutoCoder  
- NL2DB  
- 智能家居  
- 智能座舱

# 10. 智能体系统/LLM应用核心逻辑的几种分类

- CoT/简单工作流  
- 附加Function Calling的单次请求  
- 基于SOP的复杂工作流  
自规划  
…


